<table align="left">
    <tr>
        <td style="vertical-align: middle; padding-left: 0px; padding-right: 0px;">
            <a href="https://creativecommons.org/licenses/by/4.0/">
                <img src="https://licensebuttons.net/l/by/4.0/80x15.png" />
            </a>
        </td>
        <td style="vertical-align: middle; padding-left: 5px; padding-right: 0px;">
            <a href="https://opensource.org/licenses/MIT">
                <img src="https://img.shields.io/badge/License-MIT-green.svg" />
            </a>
        </td>
        <td style="vertical-align: middle; padding-left: 15px;">
            &copy; Guillaume Rongier
        </td>
    </tr>
</table>

# Using different grid types

This notebook starts from the basic example of the [first notebook](1_basic-example.ipynb) to show how to use different grid types with StratigraPy.

### Imports

Let's first import all the required packages and components:

In [ ]:
import numpy as np
from scipy.spatial import KDTree
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import cmocean

from landlab.components import FlowDirectorSteepest, FlowDirectorMFD, FlowAccumulator

from stratigrapy import RasterModelGrid, FramedVoronoiGrid, HexModelGrid
from stratigrapy.components import SeaLevelCalculator, WaterDrivenRouter

## 1. Hexagonal grid

We'll use the same case as in the [first notebook](1_basic-example.ipynb), starting with the same simulation time:

In [ ]:
timestep = 100.
runtime = 500000.
n_iterations = int(runtime/timestep)

Then we simply need to change to grid type from a raster to a hexagonal grid:

In [ ]:
grid = HexModelGrid((25, 30),
                    spacing=2500.,
                    number_of_classes=2,
                    initial_allocation=n_iterations//100 + 100,
                    number_of_layers_to_fuse=100,
                    number_of_top_layers=100,
                    fuse_continuously=True)

Defining the boundary conditions is more complicated than with a raster grid, because Landlab doesn't have a simple function like ` set_closed_boundaries_at_grid_edges` for other grid types. So we need to identify the node to close manually:

In [ ]:
is_closed = np.zeros(grid.number_of_nodes)
is_closed[
    (grid.status_at_node != grid.BC_NODE_IS_CORE) & (
        # Left border
        (grid.x_of_node < 16000.) |
        # Right border
        (grid.x_of_node > 84000) |
        # Top border
        (grid.y_of_node > 50000.)
    )
] = 1

And close them:

In [ ]:
grid.set_nodata_nodes_to_closed(is_closed.ravel(), 1)

Setting the initial elevation remain the same:

In [ ]:
elevation = grid.add_zeros('topographic__elevation', at='node', clobber=True)
elevation += 0.003*(grid.y_of_node - 50000.)

However, we cannot find the right water and sediment sources based on grid indexes, so let's use a kd-tree instead:

In [ ]:
kdtree = KDTree(grid.xy_of_node)
_, idx = kdtree.query([(grid.x_of_node.max()/2. - 2500., 50000.),
                       (grid.x_of_node.max()/2. + 2500., 50000.)])

All the steps after that remain unchanged, except for the flow director, which needs to follow the steepest path for non-raster grid:

In [ ]:
water_influx = grid.add_zeros('water__unit_flux_in', at='node', clobber=True)
water_influx[idx] = 5000. # m/yr

In [ ]:
sediment_influx = grid.add_field('sediment__unit_flux_in',
                                 np.zeros((grid.number_of_nodes, 2)),
                                 clobber=True)
sediment_influx[idx] = [0.7*50000., 0.3*50000.] # m3/yr

In [ ]:
slc = SeaLevelCalculator(grid, wavelength=[100000., 10000.], amplitude=[25., 2.5])

In [ ]:
fd = FlowDirectorSteepest(grid)
fa = FlowAccumulator(grid, flow_director=fd)

In [ ]:
wdr = WaterDrivenRouter(grid,
                        transportability_cont=[1e-8, 1e-8],
                        transportability_mar=[4e-10, 2e-10],
                        wave_base=15.,
                        max_erosion_rate_sed=1e-2,
                        max_erosion_rate_br=1e-12,
                        bedrock_composition=[0.7, 0.3],
                        fields_to_track='bathymetric__depth')

In [ ]:
for i in tqdm(range(n_iterations)):
    slc.run_one_step(timestep)
    fa.run_one_step()
    wdr.run_one_step(timestep)
    grid.stacked_layers.fuse(time=np.mean, bathymetric__depth=np.mean)
grid.stacked_layers.fuse(finalize=True, time=np.mean, bathymetric__depth=np.mean)

StratigraPy doesn't include tools to visualize the stratigraphy of non-raster grids. However, we can use Landlab's tools to visualize the final topography:

In [ ]:
grid.imshow('topographic__elevation', var_name='Elevation', var_units='m', grid_units=['m', 'm'])

And transfer the properties at the surface from the StackedLayers to the grid, for instance the fraction of the first sediment class at the surface:

In [ ]:
surface_composition = np.zeros(grid.number_of_nodes)
surface_composition[grid.core_nodes] = grid.stacked_layers.get_surface_composition()[:, 0]

_ = grid.add_field('sediment__surface_composition', surface_composition, clobber=True)

In [ ]:
grid.imshow('sediment__surface_composition',
            var_name='Fraction of the second sediment class',
            grid_units=['m', 'm'])

## 2. Voronoi grid

We can repeat the operation with the other grid types available in StratigraPy, which are modified to include a StackedLayers (Landlab's grids are not directly compatible with StratigraPy), for instance the Voronoi grid:

In [ ]:
# Define the number of iterations
timestep = 100.
runtime = 500000.
n_iterations = int(runtime/timestep)

# Define the grid
grid = FramedVoronoiGrid((25, 30),
                         xy_spacing=(2500., 2500.),
                         number_of_classes=2,
                         initial_allocation=n_iterations//100 + 100,
                         number_of_layers_to_fuse=100,
                         number_of_top_layers=100,
                         fuse_continuously=True)
is_closed = np.zeros(grid.number_of_nodes)
is_closed[
    # Left border
    ((grid.x_of_node < grid.x_of_node.min() + 1000.) & (grid.y_of_node > grid.y_of_node.min() + 1000.)) |
    # Right border
    ((grid.x_of_node > grid.x_of_node.max() - 1000.) & (grid.y_of_node > grid.y_of_node.min() + 1000.)) |
    # Top border
    (grid.y_of_node > grid.y_of_node.max() - 1000.)
] = 1
grid.set_nodata_nodes_to_closed(is_closed.ravel(), 1)

# Define the initial topography
elevation = grid.add_zeros('topographic__elevation', at='node', clobber=True)
elevation += 0.003*(grid.y_of_node - 50000.)

# Define the sources of water and sediments
kdtree = KDTree(grid.xy_of_node)
_, idx = kdtree.query([(grid.x_of_node.max()/2. - 2500., 58000.),
                       (grid.x_of_node.max()/2. + 2500., 58000.)])
water_influx = grid.add_zeros('water__unit_flux_in', at='node', clobber=True)
water_influx[idx] = 5000. # m/yr
sediment_influx = grid.add_field('sediment__unit_flux_in',
                                 np.zeros((grid.number_of_nodes, 2)),
                                 clobber=True)
sediment_influx[idx] = [0.7*50000., 0.3*50000.] # m3/yr

# Define the components for sea level variation, water flow, and sediment transport
slc = SeaLevelCalculator(grid, wavelength=[100000., 10000.], amplitude=[25., 2.5])
fd = FlowDirectorSteepest(grid)
fa = FlowAccumulator(grid, flow_director=fd)
wdr = WaterDrivenRouter(grid,
                        transportability_cont=[1e-8, 1e-8],
                        transportability_mar=[4e-10, 2e-10],
                        wave_base=15.,
                        max_erosion_rate_sed=1e-2,
                        max_erosion_rate_br=1e-12,
                        bedrock_composition=[0.7, 0.3],
                        fields_to_track='bathymetric__depth')

# Run the simulation
for i in tqdm(range(n_iterations)):
    slc.run_one_step(timestep)
    fa.run_one_step()
    wdr.run_one_step(timestep)
    grid.stacked_layers.fuse(time=np.mean, bathymetric__depth=np.mean)
grid.stacked_layers.fuse(finalize=True, time=np.mean, bathymetric__depth=np.mean)

With the same options regarding visualization:

In [ ]:
grid.imshow('topographic__elevation', var_name='Elevation', var_units='m', grid_units=['m', 'm'])

In [ ]:
surface_composition = np.zeros(grid.number_of_nodes)
surface_composition[grid.core_nodes] = grid.stacked_layers.get_surface_composition()[:, 0]

_ = grid.add_field('sediment__surface_composition', surface_composition, clobber=True)

In [ ]:
grid.imshow('sediment__surface_composition',
            var_name='Fraction of the second sediment class',
            grid_units=['m', 'm'])